In [1]:
# Import modules
# import pickle
# import h5py as h5
# import os
# import sys
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib as mpl
# import seaborn as sns
# import netCDF4 as nc
# import itertools
import datetime
import time
from scipy.io import loadmat
# import statsmodels.api as sm

In [14]:
def decyeartodatetime(din):
    start = din
    year = int(start)
    rem = start - year
    base = datetime.datetime(year, 1, 1)
    result = base + datetime.timedelta(seconds=(base.replace(year=base.year + 1) - base).total_seconds() * rem)
#     if result.hour>12:
#         result = result+dt.timedelta(days=1)
#     result2 = result.replace(hour=0, minute=0, second=0, microsecond=0)
    result2 = result.replace(minute=0, second=0, microsecond=0)
    return result2

In [3]:
#### CHANGE THE PATH TO POINT AT THESE FILES
fsmb = loadmat('relayerthinningpaper/SMB_MERRA_atpix.mat')
ftskin = loadmat('relayerthinningpaper/TEMP_MERRA_atpix.mat')
ftime = loadmat('relayerthinningpaper/TIME_MERRA-2_Mod.mat')

In [27]:
merra_time = ftime['time']
smb = fsmb['smb']
tskin = ftskin['temp']
dti = [decyeartodatetime(ii) for ii in merra_time.flatten()]
M_df = pd.DataFrame({'decdate':merra_time.flatten(),'smb':smb.flatten(),'tskin':tskin.flatten()},index=dti)
i2019 = np.where(M_df.index.year==2019)[0]
M_df.drop(M_df.iloc[i2019].index,inplace=True) #get rid of 2019 data

In [65]:
M_df['smb_pertime'] = M_df['smb']*5/365.25 # just checking that we get the same answer when we take the mean and sum

In [68]:
annual_df = M_df.resample('a').agg({'decdate':'mean','smb':'mean','tskin':'mean','smb_pertime':'sum'})

In [80]:
annual_df

,decdate,smb,tskin,smb_pertime
1980-12-31,1980.498577,0.793930,256.879265,0.793386
1981-12-31,1981.497203,0.864710,255.652457,0.864118
1982-12-31,1982.497203,0.750207,255.450740,0.749693
1983-12-31,1983.497203,0.657285,255.074400,0.656835
1984-12-31,1984.495845,0.732440,255.337704,0.731939
1985-12-31,1985.494464,0.967989,256.170298,0.967327
1986-12-31,1986.494463,0.495310,253.922535,0.494971
1987-12-31,1987.494463,0.581775,254.690308,0.581376
1988-12-31,1988.499943,0.874415,256.908809,0.885787
1989-12-31,1989.505422,0.532248,255.435862,0.531883


In [81]:
spinuptime = 600 # years you want to spin up for 
data_timestart = 1980
data_timeend = 2018
modeltimestart = data_timestart-spinuptime
modeltime = np.arange(modeltimestart,data_timeend+1)

In [82]:

smb_mean = annual_df.smb['1980':'2007'].mean()
smb_sd = annual_df.smb['1980':'2007'].std()
spin_smb = np.random.normal(smb_mean,smb_sd,spinuptime) #generate a random SMB for each year of the spin up
smb_full = np.concatenate((spin_smb,annual_df.smb.values))
smb_out = np.array((modeltime,smb_full))

tskin_mean = annual_df.tskin['1980':'2007'].mean()
tskin_sd = annual_df.tskin['1980':'2007'].std()
spin_tskin = np.random.normal(tskin_mean,tskin_sd,spinuptime) #generate a random SMB for each year of the spin up
tskin_full = np.concatenate((spin_tskin,annual_df.tskin.values))
tskin_out = np.array((modeltime,tskin_full))

np.savetxt('smb_out.csv',smb_out,delimiter=',',fmt='%1.4f')
np.savetxt('tskin_out.csv',tskin_out,delimiter=',',fmt='%1.4f')